# Phase 4 - Efficiency
If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [ ]:
import os
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

This is the object model for receiving questions.

In [ ]:
from enum import Enum
from pydantic import BaseModel

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    popular_choice = "popular_choice"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Get the number of tokens
tiktoken is a library which allows you to get the number of tokens. 
Ensure you pick the correct encoding for your model based on this list. https://github.com/openai/tiktoken/blob/c0ba74c238d18b4824c25f3c27fc8698055b9a76/tiktoken/model.py#L20


In [ ]:
def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    """Returns the number of tokens in a text by a given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

number_of_tokens=get_num_tokens_from_string("Hello, Azure AI Adventure Day!")
print(f"Number of tokens in the string: {number_of_tokens}")


# YOUR Mission: 
Ensure the answers provided are correct and you don't return info you shouldn't return.


In [ ]:
async def ask_question(ask: Ask):
    """
    Ask a question
    """

    # Send a completion call to generate an answer
    print('Sending a request to openai')
    start_phrase = ask.question
    messages=  [{"role" : "assistant", "content" : start_phrase}]
    number_of_tokens_in= get_num_tokens_from_string(messages[0]['content']);
    
    response = client.chat.completions.create(
        model = deployment_name,
        messages =messages,
    )

    number_of_tokens_out= get_num_tokens_from_string(response.choices[0].message.content);

    print(f"Number of tokens in the input: {number_of_tokens_in}")
    print(f"Number of tokens in the output: {number_of_tokens_out}")
    print('Total Tokens Used: ' + str(number_of_tokens_in + number_of_tokens_out))

    answer = Answer(answer=response.choices[0].message.content)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens

    return answer

Use this snippet to try your method with several questions.

In [ ]:

ask = Ask(question="What is a liquid humans drink (and Smorghs hate)?", type=QuestionType.popular_choice)
answer = await ask_question(ask)
print('Answer:', answer)

Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase5 $AZURE_ENV_NAME
```
Make sure to provide the URL of your endpoint in the team portal!